In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
# Custom float formatter
pd.set_option('display.float_format', lambda x: f"{x:,.2f}")

class Portfolio:
    def __init__(self, tickers, total_investment):
        self.tickers = tickers
        self.total_investment = total_investment
        self.market_caps, self.current_prices, self.valid_tickers = self.get_market_caps_and_prices()
        self.historical_data = self.get_historical_data()
        self.tickers, self.market_caps = self.apply_constraints()
        self.combined_weights = self.calculate_combined_weights()
        self.portfolio_df = self.initialize_positions()
        self.all_tickers_data = self.fetch_ticker_data()

    def get_market_caps_and_prices(self):
        market_caps = {}
        current_prices = {}
        valid_tickers = []
        for ticker in self.tickers:
            try:
                stock = yf.Ticker(ticker)
                market_cap = stock.info.get('marketCap', None)
                current_price = stock.history(period='1d')['Close'].iloc[-1]
                if market_cap is not None and not np.isnan(current_price):
                    market_caps[ticker] = market_cap
                    current_prices[ticker] = current_price
                    valid_tickers.append(ticker)
            except Exception as e:
                print(f"Error fetching data for {ticker}: {e}")
        return market_caps, current_prices, valid_tickers

    def get_historical_data(self):
        try:
            historical_data = yf.download(self.valid_tickers, period='5y')['Adj Close']
        except Exception as e:
            print(f"Error fetching historical data: {e}")
            historical_data = pd.DataFrame()
        return historical_data

    def apply_constraints(self, min_market_cap=1e8):
        filtered_tickers = {ticker: market_cap for ticker, market_cap in self.market_caps.items() if market_cap and market_cap >= min_market_cap}
        return list(filtered_tickers.keys()), filtered_tickers

    def fetch_ticker_data(self):
        all_tickers_data = {}
        for ticker_symbol in self.tickers:
            ticker = yf.Ticker(ticker_symbol)
            info = ticker.info
            financials = ticker.financials
            earnings = ticker.income_stmt.get('Net Income', pd.Series())
            dividends = ticker.dividends
            stats = {
                'P/E Ratio': info.get('trailingPE'),
                'PEG Ratio': info.get('pegRatio'),
                'Beta': info.get('beta'),
                'Volume': info.get('volume'),
                'EBITDA': info.get('ebitda'),
                'ROE': info.get('returnOnEquity'),
                'ROA': info.get('returnOnAssets')
            }
            historical_data = ticker.history(period="1y")
            all_tickers_data[ticker_symbol] = {
                'info': info,
                'financials': financials,
                'earnings': earnings,
                'dividends': dividends,
                'stats': stats,
                'historical_data': historical_data
            }
        return all_tickers_data

    def market_cap_weighted(self):
        total_market_cap = sum(self.market_caps.values())
        return {ticker: market_cap / total_market_cap for ticker, market_cap in self.market_caps.items()}

    def score_tilt_weighted(self):
        returns = self.historical_data.pct_change().mean()
        scores = returns.rank(ascending=False)
        total_score = scores.sum()
        return {ticker: scores[ticker] / total_score for ticker in self.tickers}

    def score_weighted(self):
        volatilities = self.historical_data.pct_change().std()
        scores = 1 / volatilities
        scores = scores.rank(ascending=False)
        total_score = scores.sum()
        return {ticker: scores[ticker] / total_score for ticker in self.tickers}

    def equal_weighted(self):
        num_assets = len(self.tickers)
        return {ticker: 1 / num_assets for ticker in self.tickers}

    def inverse_volatility_weighted(self):
        volatilities = self.historical_data.pct_change().std()
        inv_vol = 1 / volatilities
        total_inv_vol = inv_vol.sum()
        return {ticker: inv_vol[ticker] / total_inv_vol for ticker in self.tickers}

    def minimum_correlation_weighted(self):
        corr_matrix = self.historical_data.pct_change().corr()
        inv_corr_sum = 1 / corr_matrix.sum()
        total_inv_corr = inv_corr_sum.sum()
        return {ticker: inv_corr_sum[ticker] / total_inv_corr for ticker in self.tickers}

    def dividend_yield_weighted(self):
        dividend_yields = {}
        for ticker in self.tickers:
            prices = self.historical_data[ticker].to_numpy()
            pct_changes = np.diff(prices) / prices[:-1]
            mean_pct_change = np.mean(pct_changes)
            if self.current_prices[ticker]:
                dividend_yield = mean_pct_change / self.current_prices[ticker]
                dividend_yields[ticker] = dividend_yield
        total_yield = np.sum(list(dividend_yields.values()))
        return {ticker: dividend_yields[ticker] / total_yield for ticker in dividend_yields}

    def momentum_weighted(self):
        momentum = (self.historical_data.iloc[-1] - self.historical_data.iloc[0]) / self.historical_data.iloc[0]
        total_momentum = momentum.sum()
        return {ticker: momentum[ticker] / total_momentum for ticker in self.tickers}

    def earnings_yield_weighted(self):
        earnings_yield = np.array([1 / self.market_caps[ticker] for ticker in self.tickers])
        total_earnings_yield = np.sum(earnings_yield)
        return {ticker: earnings_yield[i] / total_earnings_yield for i, ticker in enumerate(self.tickers)}

    def volatility_trend_weighted(self):
        prices = self.historical_data[self.tickers].to_numpy()
        pct_changes = np.diff(prices, axis=0) / prices[:-1, :]
        window = 30
        num_periods, num_tickers = pct_changes.shape
        rolling_volatility = np.empty((num_periods - window + 1, num_tickers))
        for i in range(num_tickers):
            rolling_volatility[:, i] = np.array([
                np.std(pct_changes[j:j + window, i]) for j in range(num_periods - window + 1)])
        mean_rolling_volatility = np.mean(rolling_volatility, axis=0)
        inv_rolling_vol = 1 / mean_rolling_volatility
        total_inv_rolling_vol = np.sum(inv_rolling_vol)
        return {ticker: inv_rolling_vol[i] / total_inv_rolling_vol for i, ticker in enumerate(self.tickers)}

    def value_weighted(self):
        pe_ratios = np.array([
            self.market_caps[ticker] / self.current_prices[ticker] for ticker in self.tickers])
        total_pe = np.sum(pe_ratios)
        return {ticker: pe_ratios[i] / total_pe for i, ticker in enumerate(self.tickers)}

    def calculate_combined_weights(self):
        market_cap_weights = self.market_cap_weighted()
        score_tilt_weights = self.score_tilt_weighted()
        score_weights = self.score_weighted()
        equal_weights = self.equal_weighted()
        inverse_vol_weights = self.inverse_volatility_weighted()
        min_corr_weights = self.minimum_correlation_weighted()
        dividend_yield_weights = self.dividend_yield_weighted()
        momentum_weights = self.momentum_weighted()
        earnings_yield_weights = self.earnings_yield_weighted()
        volatility_trend_weights = self.volatility_trend_weighted()
        value_weights = self.value_weighted()

        factor_returns = {
            'market_cap': np.mean(list(market_cap_weights.values())),
            'score_tilt': np.mean(list(score_tilt_weights.values())),
            'score': np.mean(list(score_weights.values())),
            'equal': np.mean(list(equal_weights.values())),
            'inverse_vol': np.mean(list(inverse_vol_weights.values())),
            'min_corr': np.mean(list(min_corr_weights.values())),
            'dividend_yield': np.mean(list(dividend_yield_weights.values())),
            'momentum': np.mean(list(momentum_weights.values())),
            'earnings_yield': np.mean(list(earnings_yield_weights.values())),
            'volatility_trend': np.mean(list(volatility_trend_weights.values())),
            'value': np.mean(list(value_weights.values()))
        }
        total_factor_return = sum(factor_returns.values())

        factor_scores = {key: value / total_factor_return for key, value in factor_returns.items()}

        combined_weights = {}
        for ticker in self.tickers:
            combined_weights[ticker] = (
                (market_cap_weights[ticker] * factor_scores['market_cap']) +
                (score_tilt_weights[ticker] * factor_scores['score_tilt']) +
                (score_weights[ticker] * factor_scores['score']) +
                (equal_weights[ticker] * factor_scores['equal']) +
                (inverse_vol_weights[ticker] * factor_scores['inverse_vol']) +
                (min_corr_weights[ticker] * factor_scores['min_corr']) +
                (dividend_yield_weights[ticker] * factor_scores['dividend_yield']) +
                (momentum_weights[ticker] * factor_scores['momentum']) +
                (earnings_yield_weights[ticker] * factor_scores['earnings_yield']) +
                (volatility_trend_weights[ticker] * factor_scores['volatility_trend']) +
                (value_weights[ticker] * factor_scores['value'])
            )
        return combined_weights

    def initialize_positions(self):
        initial_positions = {ticker: self.combined_weights[ticker] * self.total_investment for ticker in self.tickers}
        portfolio_df = pd.DataFrame(list(initial_positions.items()), columns=['Ticker', 'Initial_Position'])
        portfolio_df['Current_Price'] = portfolio_df['Ticker'].map(self.current_prices)
        portfolio_df['Shares'] = portfolio_df['Initial_Position'] / portfolio_df['Current_Price']
        portfolio_df['Weight_Percentage'] = portfolio_df['Ticker'].map(self.combined_weights) * 100
        portfolio_df = portfolio_df.sort_values(by='Weight_Percentage', ascending=False)
        return portfolio_df

    def rebalance_portfolio(self):
        daily_positions = []
        daily_trades = []
        portfolio_performance = []
        for date in self.historical_data.index:
            daily_prices = self.historical_data.loc[date]
            current_prices = np.array([daily_prices[ticker] for ticker in self.portfolio_df['Ticker']])
            current_values = self.portfolio_df['Shares'].to_numpy() * current_prices
            total_value = np.sum(current_values)
            target_weights = self.portfolio_df['Weight_Percentage'].to_numpy() / 100
            target_values = target_weights * total_value
            new_shares = target_values / current_prices
            trades = new_shares - self.portfolio_df['Shares'].to_numpy()
            self.portfolio_df['Shares'] = new_shares

            daily_positions.append(self.portfolio_df[['Ticker', 'Shares']].copy())
            daily_trades.append(np.column_stack((self.portfolio_df['Ticker'], trades)))
            total_return = (total_value - self.total_investment) / self.total_investment
            trading_costs = np.sum(np.abs(trades)) * 0.001
            net_return = (total_value - self.total_investment - trading_costs) / self.total_investment
            portfolio_performance.append({'Date': date, 'Total_Value': total_value, 'Total_Return': total_return, 'Net_Return': net_return})
        return daily_positions, daily_trades, portfolio_performance

    def display_portfolio(self):
        print(self.portfolio_df)

In [3]:
# Instantiate Portfolio Class and Run Rebalance Function
tickers = [
    'AAPL', 'NVDA', 'MSFT', 'GOOGL', 'AMZN', 'META', 'BRK-A', 'TSM', 'LLY', 'AVGO',
    'TSLA', 'WMT', 'JPM', 'UNH', 'XOM', 'V', 'ORCL', 'MA', 'HD', 'PG',
    'NVO', 'COST', 'JNJ', 'ABBV', 'ASML', 'BAC', 'NFLX', 'KO', 'MRK', 'CVX',
    'CRM', 'AMD', 'SAP', 'BABA', 'TMUS', 'ACN', 'PEP', 'AZN', 'NVS', 'TM',
    'TMO', 'LIN', 'MCD', 'ADBE', 'CSCO', 'IBM', 'WFC', 'GE', 'ABT', 'PDD'
]
total_investment = 10000000000

portfolio = Portfolio(tickers, total_investment)
daily_positions, daily_trades, portfolio_performance = portfolio.rebalance_portfolio()
portfolio.display_portfolio()

[*********************100%***********************]  50 of 50 completed


   Ticker  Initial_Position  Current_Price       Shares  Weight_Percentage
1    NVDA    590,863,200.76         128.91 7,296,659.72               5.91
0    AAPL    321,943,614.20         248.05 2,066,161.90               3.22
10   TSLA    320,052,691.74         440.13 1,157,615.35               3.20
3   GOOGL    268,218,210.33         188.40 2,266,371.74               2.68
4    AMZN    257,626,244.02         220.52 1,859,798.45               2.58
2    MSFT    253,625,215.38         437.39   923,096.99               2.54
9    AVGO    250,494,030.28         223.62 1,783,242.96               2.50
49    PDD    246,765,199.14         101.35 3,876,001.50               2.47
25    BAC    235,453,837.20          43.50 8,616,686.39               2.35
11    WMT    229,730,635.33          93.55 3,909,298.91               2.30
20    NVO    222,676,717.16         105.96 3,345,466.11               2.23
7     TSM    207,732,003.39         195.56 1,691,013.82               2.08
46    WFC    203,265,783.

In [4]:
for performance in portfolio_performance[-10:]:
    print(f"\nDate: {performance['Date']}\n Total Value: {performance['Total_Value']:.2f}\n Total Return: {performance['Total_Return']:.2%}\n Net Return: {performance['Net_Return']:.2%}")



Date: 2024-12-05 00:00:00
 Total Value: 16365732568.77
 Total Return: 63.66%
 Net Return: 63.66%

Date: 2024-12-06 00:00:00
 Total Value: 16452912412.13
 Total Return: 64.53%
 Net Return: 64.53%

Date: 2024-12-09 00:00:00
 Total Value: 16377903400.25
 Total Return: 63.78%
 Net Return: 63.78%

Date: 2024-12-10 00:00:00
 Total Value: 16287853058.30
 Total Return: 62.88%
 Net Return: 62.88%

Date: 2024-12-11 00:00:00
 Total Value: 16397265124.23
 Total Return: 63.97%
 Net Return: 63.97%

Date: 2024-12-12 00:00:00
 Total Value: 16274148906.56
 Total Return: 62.74%
 Net Return: 62.74%

Date: 2024-12-13 00:00:00
 Total Value: 16321967748.72
 Total Return: 63.22%
 Net Return: 63.22%

Date: 2024-12-16 00:00:00
 Total Value: 16348898988.70
 Total Return: 63.49%
 Net Return: 63.49%

Date: 2024-12-17 00:00:00
 Total Value: 16330447399.43
 Total Return: 63.30%
 Net Return: 63.30%

Date: 2024-12-18 00:00:00
 Total Value: 15919292816.94
 Total Return: 59.19%
 Net Return: 59.19%


In [5]:
for i, daily_trade in enumerate(daily_trades[-10:]):
    print(f"\nTrade Date: {portfolio_performance[-10 + i]['Date']}\n", pd.DataFrame(daily_trade, columns=['Ticker', 'Trade Amount']))


Trade Date: 2024-12-05 00:00:00
    Ticker Trade Amount
0    NVDA     6,292.06
1    AAPL       733.48
2    TSLA   -45,108.33
3   GOOGL    26,418.97
4    AMZN   -20,050.75
5    MSFT   -10,709.93
6    AVGO     2,378.71
7     PDD   -10,095.51
8     BAC  -108,152.05
9     WMT   -33,741.06
10    NVO     6,154.04
11    TSM   -29,150.95
12    WFC   -34,986.54
13    AZN   -52,657.18
14   CSCO   -34,307.43
15     KO   -56,608.64
16    AMD    42,971.61
17    XOM   -10,962.50
18     GE    98,040.93
19   ORCL    18,117.86
20    LLY     2,065.77
21    NVS   -18,335.03
22    MRK   -61,918.04
23   META     4,103.45
24    ABT     9,536.93
25    CVX   -10,787.32
26   ABBV     3,432.16
27    PEP    -2,652.00
28     PG    -7,789.45
29    JNJ    12,963.39
30   TMUS    -3,590.43
31    JPM    -9,268.47
32   BABA    31,735.88
33    IBM    -5,815.37
34     TM     4,168.78
35    MCD   -13,207.00
36    SAP    -7,429.81
37    TMO     7,952.04
38      V     2,651.42
39   ADBE    -1,353.13
40    ACN     4,174.21
